In [ ]:
import os
import glob

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

import matplotlib.pyplot as plt

# create image dict

In [ ]:
images = glob.glob('/root/data/headtail/frames/*/*.jpg') + ['/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010008_rectified/right_sotra-small-pen_0_1538489067394.jpg']
image_dic = {}
for image in images: 
    image_dic[os.path.basename(image)] = image

# load model

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
model = get_unet(3, 512, 512, classes=2)

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

# load data and define generators

In [ ]:
import glob
import cv2
import numpy as np
import random
import math
import json
SEED = 448
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import shutil
from collections import Counter
from PIL import Image, ImageDraw

In [ ]:
annotations = json.load(open("./head_tails_segmentation.json"))

In [ ]:
annotations = [ann for ann in annotations if ann['Label'] != 'Skip']

In [ ]:
tmp = [ann for ann in annotations if "erko" in ann['External ID']]

In [ ]:
def create_mask_from_json(ann, input_size=(512, 512)):
    """from json file create the mask and load the image
    outputs: 
    - image: RGB
    - mask: (H, W, 2)"""
    img = Image.open(image_dic[ann['External ID']])
    width, height = img.size
    image = np.array(img.resize(input_size))
    
    # create head mask
    if 'Head' in ann['Label']:
        head_labels = ann['Label']['Head']
        head_mask_img = Image.new('L', (width, height), 0)
        for hl in head_labels:
            geometry = hl['geometry']
            polygon = [(k['x'], k['y']) for k in geometry]
            ImageDraw.Draw(head_mask_img).polygon(polygon, outline=1, fill=1)
            head_mask = cv2.resize(np.array(head_mask_img), input_size)
    else:
        head_mask = np.zeros(input_size)
    
    # create tail mask
    if 'Tail' in ann['Label']:
        tail_labels = ann['Label']['Tail']
        tail_mask_img = Image.new('L', (width, height), 0)
        for tl in tail_labels:
            geometry = tl['geometry']
            polygon = [(k['x'], k['y']) for k in geometry]
            ImageDraw.Draw(tail_mask_img).polygon(polygon, outline=1, fill=1)
            tail_mask = cv2.resize(np.array(tail_mask_img), input_size)
    else:
        tail_mask = np.zeros(input_size)
    mask = np.stack([head_mask, tail_mask], axis=2)
    return image, mask

#### quick test

In [ ]:
img, mask = create_mask_from_json(tmp[0])

#### Creating the generator now

In [ ]:
from albumentations import *

In [ ]:
random.seed(1356)
random.shuffle(annotations)
cutoff = int(len(annotations)*0.8)
trainset = annotations[:cutoff]
valset = annotations[cutoff:]

In [ ]:
batch_size = 8
input_size = (512, 512, 3)
steps_per_epoch = len(trainset) // batch_size
steps_per_epoch_val = len(valset) // batch_size

In [ ]:
def generator(dataset, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    img_size = input_shape[0]
    aug = Compose([HorizontalFlip(p=0.5), 
              VerticalFlip(p=0.5),
              RandomRotate90(p=0.5),
              RandomSizedCrop(p=0.3, min_max_height=(400, 512), height=512, width=512),
              Transpose(p=0.5)])
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 2), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            image, mask = create_mask_from_json(dataset[j], (input_shape[0], input_shape[1]))
            augmented = aug(image=image, mask=mask)
            x_batch[ind, ...] = np.expand_dims(augmented['image'], axis=0)
            y_batch[ind, ...] = np.expand_dims(augmented['mask'], axis=0)
        
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
train_generator = generator(trainset, steps_per_epoch, batch_size, input_size)
val_generator = generator(valset, steps_per_epoch_val, batch_size, input_size)

In [ ]:
xb, yb = next(train_generator)

#### some viz to check

In [ ]:
for i in range(8):
    plt.imshow(xb[i,...])
    plt.imshow(yb[i,...,0], alpha=0.3)
    plt.imshow(yb[i,...,1], alpha=0.3)
    plt.show()

## Train

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = lr
    drop = 0.5
    epochs_drop = 20.0
    fake_epoch = epoch
    lrate = initial_lrate * math.pow(drop, math.floor((1+fake_epoch)/epochs_drop))
    print('lr {}'.format(lrate))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# save model
filepath = os.path.join('/root/data/models/headtail/', '1004_model_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_jaccard_coef_int', 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)
saveh = SaveHistory('./headtail_history2.json')

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        verbose=1,
        callbacks=[saveh, lr_scheduler, checkpoint],
        validation_data= val_generator,
        validation_steps= steps_per_epoch_val, initial_epoch=50)

# FORWARD PASS

In [ ]:
import glob
from pycocotools.mask import encode

In [ ]:
weights_path = '/root/data/models/headtail/1004_model_62.h5'

In [ ]:
model.load_weights(weights_path)

In [ ]:
base_dir = '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/'
experiences = os.listdir(base_dir)
all_image_path = []
for experience in experiences:
    folder_path = os.path.join(base_dir, experience)
    if not os.path.isdir(folder_path):
        continue
    if "rectified" not in folder_path:
        continue
    if "reference" in folder_path:
        continue
    print(folder_path)
    all_image_path += glob.glob(folder_path + '/*.jpg')

In [ ]:
results = []
for path in all_image_path[:6]:
    image = np.expand_dims(np.array(Image.open(path).resize((512, 512))), axis=0)
    predictions = model.predict(image).squeeze()
    predictions = np.array(predictions, dtype=np.uint8)
    tmp = {'image_path': path, 'mask': encode(np.asfortranarray(predictions))}

In [ ]:
plt.imshow(image.squeeze())
plt.imshow(predictions[...,1], alpha=0.3)